In [1]:
!pip3 install -r requirements.txt --quiet

In [2]:
import boto3
import json

import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from getpass import getpass
from langchain.agents import create_pandas_dataframe_agent
from pandasai import SmartDataframe

import torch
from transformers import (
    pipeline,
    AutoModelForCausalLM,
    AutoTokenizer,
)
from langchain.llms import HuggingFacePipeline
from datetime import date, time

import warnings
from IPython.display import Markdown

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/torch/cuda/__init__.py:551: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


### <a name="#2">Define the Bedrock Model for Inference</a>
(<a href="#0">Go to top</a>)


Let's select the Bedrock model the same way we did in the previous labs.

**Please opt for frugal practices when using Bedrock such as using smaller LLMs for simpler tasks and only reserving the use of the larger LLMs for more complex use cases.**

In [4]:
from langchain.llms.bedrock import Bedrock
from ml_utils.helpers import validate_inference_parameters

# Define the bedrock client
bedrock = boto3.client(
    service_name="bedrock",
    region_name="us-east-1",
    endpoint_url="https://bedrock.us-east-1.amazonaws.com",
)

# Define the bedrock-runtime client that will be used for predictions
bedrock_runtime = boto3.client(service_name="bedrock-runtime")


##########################################################################
########################### Select the LLM ###############################

# bedrock_model_id = "ai21.j2-mid-v1"
# bedrock_model_id = "cohere.command-text-v14"
bedrock_model_id = "ai21.j2-ultra-v1"
# bedrock_model_id = 'anthropic.claude-v2'

##########################################################################
##########################################################################

# Each model has a different set of inference parameters
inference_modifier = {
    "maxTokens": 4096,
    "temperature": 0.0,
    "topP": 1,
}

# Sanity check for the inference parameters
validate_inference_parameters(
    model_id=bedrock_model_id, inference_parameters=inference_modifier
)

# Define the langchain module with the selected bedrock model
bedrock_llm = Bedrock(
    model_id=bedrock_model_id, client=bedrock_runtime, model_kwargs=inference_modifier
)

In [5]:
df = pd.read_csv('data/titanic.csv')
df.head()
sdf = SmartDataframe(df, config={"llm": bedrock_llm})
response = sdf.chat('Calculate the median age')
print(response)

27.0
